# Routing Optimization Capstone Project  - optimizaiton material generation pipeline

Project student team: Peter Pan; Vincent Pan; Sanjit Sokhi, Jerry Wang, Jiadi Zhang

Advisor: Amr Farahat

Creation date: 2023-04-01

This notebook generates the necessary arcs and nodes for the network flow optimization base upon 1) drivers' home location, 2) jobs' origin and destination location, 3) the time windows for pickups and deliveries. The next jupyter notebook for the optimization model reads the materials generated by this notebook and performs the optimization.
To protect the company's information, the data used in this notebook is completed simulated.
The materials are generated in the form of a pickle file. 
You can glimpse the sample materials at the bottom of the page

Choose the number of drivers and number of orders you want

In [596]:
num_driver =  30
num_order = 150
random_state_param = 886
#mip_gap = 0.05  

 ## Importing packages

In [597]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime, timedelta
import numpy as np
from geopy.geocoders import Nominatim 
from geopy import distance 
from multidict import MultiDict
import math
import gurobipy as gp
from gurobipy import GRB

 ## Setting functions

Distance functions to calculate the distance between two areas

In [598]:
def my_distance(origin_zip,dest_zip):
    geolocator = Nominatim(user_agent = "my_app_name")
    origin_loc = geolocator.geocode(origin_zip,country_codes ='us')
    dest_loc = geolocator.geocode(dest_zip,country_codes ='us')
    origin_latlon = (origin_loc.latitude,origin_loc.longitude)
    dest_latlon = (dest_loc.latitude,dest_loc.longitude)
    return (distance.distance(origin_latlon,dest_latlon).miles)


In [599]:
def haversine_distance(lon1, lat1, lon2, lat2):
    try:
        R = 6371  # radius of the Earth in kilometers
        dlon = math.radians(lon2 - lon1)
        dlat = math.radians(lat2 - lat1)
        a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c /1.609
    except:
        return None

Time function to calcualte the time consumption for a trucker to travel certain dsitance

In [600]:
def my_time(dis):
    speed = 7.6 * math.log(dis) + 4.5
    time_consumption = dis/speed
    return time_consumption

Function to standardize zip codes

In [601]:
def zip_correcter(zipcodes):
    zipdf = zipcodes.copy()
    corrected_zips = []
    for azip in zipdf:
        zip_length = len(str(azip))
        if zip_length < 5:
            corrected_zips.append((5-zip_length)*'0' + str(azip))
        elif zip_length > 5:
            corrected_zips.append(str(azip)[:5])
        else:
            corrected_zips.append(azip)
    return corrected_zips

 ## importing and processing data

Data import section

In [602]:
df = pd.read_csv("C:/Users/jiadiz/Desktop/PGT Trucking/initial model/Github/simulated_order_sample.csv",dtype={'dest_zipcode':str,'origin_zipcode':str})
df['dest_zipcode'] = zip_correcter(df['dest_zipcode'])
df['origin_zipcode'] = zip_correcter(df['origin_zipcode'])

In [603]:
df2 = df.dropna().sample(num_order, random_state = random_state_param)

In [604]:
df2['origin_zipcode'] = df2['origin_zipcode'].astype(str)
df2['dest_zipcode'] = df2['dest_zipcode'].astype(str)

Creating pickup_midpoint

In [605]:
#Adding the columns pick_up_midpoint
from datetime import datetime
df2['pickup_start_date_time'] = df2.apply(lambda row: datetime.strptime(row['pickup_window_start'], '%m/%d/%Y %H:%M'),axis = 1)
df2['pickup_end_date_time'] = df2.apply(lambda row: datetime.strptime(row['pickup_window_end'], '%m/%d/%Y %H:%M'),axis = 1)
df2['pickup_midpoint'] = df2.apply(lambda row: row['pickup_start_date_time'] + (row['pickup_end_date_time'] - row['pickup_start_date_time']) / 2,axis = 1)

Creating Delivery_midpoint time

In [606]:
#Adding the columns delivery midpoints
df2['delivery_start_date_time'] = df2.apply(lambda row: datetime.strptime(row['delivery_window_start'], '%m/%d/%Y %H:%M'),axis = 1)
df2['delivery_end_date_time'] = df2.apply(lambda row: datetime.strptime(row['delivery_window_end'], '%m/%d/%Y %H:%M'),axis = 1)
df2['delivery_midpoint'] = df2.apply(lambda row: row['delivery_start_date_time'] + (row['delivery_end_date_time'] - row['delivery_start_date_time']) / 2,axis = 1)

Assuming that the drivers start working at 6am

In [607]:
mondays = []
import datetime
#Computing the mondays 6am of the weeks that the jobs are in
for ts in df2['pickup_midpoint']:
    # convert to datetime object
    dt = ts.to_pydatetime()

    # get the date and weekday
    date = dt.date()
    weekday = date.weekday()

    # calculate timedelta to previous Monday
    days_to_subtract = (7 + weekday) % 7
    monday = date - datetime.timedelta(days=days_to_subtract)

    # set hour and minute
    monday = datetime.datetime.combine(monday, datetime.time(hour=6, minute=0))

    # convert back to timestamp object
    monday = pd.Timestamp(monday)

    mondays.append(monday)

mondays  # output: [Timestamp('2023-03-06 06:00:00'),

df2['monday_6am'] = mondays

In [608]:
def time_to_arrive(mid_point):
    time_available = max(mid_point.weekday()-1, 0)*12 + mid_point.hour-6 + mid_point.minute/60
    return time_available
    

Computing whether the driver could arrive to the job if they begin driving at 6am

In [609]:
#computing the time difference between monday 6am and the pickup_midpoint time
df2['driver_home_to_job_midpoint_time_limit'] = df2.apply(lambda row : time_to_arrive(row['pickup_midpoint']),axis = 1)

Importing drivers

In [610]:
#cross merging driver sample and job dataframe
md = pd.read_csv('C:/Users/jiadiz/Desktop/PGT Trucking/initial model/Github/simulated_driver_sample.csv', dtype={'depot_zipcode':str})

In [611]:
md['depot_zipcode'] = zip_correcter(md['depot_zipcode'])
ds = md.sample(num_driver, random_state = random_state_param)
ds['key']=1
df2['key']=1

Creating terminals

In [612]:
#assigning terminals to drivers
terminals = []

for i in range (0,len(ds)):
    terminals.append(str('T'+str(i)))

ds['terminals'] = terminals

Merging drivers and orders to compute all driver-to-order possibilities

In [613]:
big_frame = pd.merge(ds,df2, on = 'key')

In [614]:
zips = pd.read_csv('uszips.csv', dtype={'zip':str})
zips['zip'] = zip_correcter(zips['zip'])
zips['zip'] = zips['zip'].astype(str)

Examine how many driver zip codes are not included in the zip code profile

In [615]:
len(zips['zip'][zips['zip'].isin(big_frame['depot_zipcode'])]) - len(big_frame['depot_zipcode'].unique())

0

In [616]:
big_frame['origin_zipcode'] = big_frame['origin_zipcode'].astype(str)
big_frame['dest_zipcode'] = big_frame['dest_zipcode'].astype(str)

Examine how many order zip codes are not included in the zip code profile

In [617]:
#zips['zip'][zips['zip'].isin(big_frame['origin_zipcode'])]
len(zips['zip'][zips['zip'].isin(big_frame['origin_zipcode'])]) - len(big_frame['origin_zipcode'].unique())

0

In [618]:
len(zips['zip'][zips['zip'].isin(big_frame['dest_zipcode'])]) - len(big_frame['dest_zipcode'].unique())

0

In [619]:
big_frame['dest_zipcode'][~big_frame['dest_zipcode'].isin(zips['zip'])].unique()

array([], dtype=object)

In [620]:
zips.columns = ['zip','depot_lat','depot_lng']
bigger_frame = pd.merge(zips,big_frame, left_on = 'zip', right_on = 'depot_zipcode')
zips.columns = ['zip','origin_lat','origin_lng']
bigger_bigger_frame = pd.merge(zips,bigger_frame, left_on = 'zip', right_on = 'origin_zipcode')
zips.columns = ['zip','dest_lat','dest_lng']
biggest_frame = pd.merge(zips, bigger_bigger_frame, left_on = 'zip', right_on = 'dest_zipcode')

Computing driver to job distance

In [621]:
biggest_frame['driver_to_job_distance'] = biggest_frame.apply(lambda row: haversine_distance(row['depot_lng'],row['depot_lat'],row['origin_lng'],row['origin_lat']),axis = 1)

In [622]:
distance_verification_sample = biggest_frame.sample(300)

Examining the validity of haversine distance. The examination costs time, so let the codes be commented out if not examining. 

In [623]:
import seaborn as sns

In [624]:
biggest_frame['haversine_distance'] = biggest_frame.apply(lambda row: haversine_distance(row['origin_lng'],row['origin_lat'],row['dest_lng'],row['dest_lat']),axis=1)

In [625]:
exam = pd.DataFrame((biggest_frame['haversine_distance'].drop_duplicates()-biggest_frame['DispatchMiles'].drop_duplicates())/biggest_frame['DispatchMiles'].drop_duplicates())

In [626]:
exam.dropna().mean()

0    19.925135
dtype: float64

In [627]:
biggest_frame['driver_to_job_distance_plus'] = biggest_frame['driver_to_job_distance'] + biggest_frame['haversine_distance']

In [628]:
biggest_frame['driver_to_job_time'] = biggest_frame['driver_to_job_distance_plus'] .apply(lambda row: my_time(row))

In [629]:
#all the feasible 1st jobs. If a job cannot be done on the first wave of driver's after 6am on Monday, the jobs cannot be done
first_wave = biggest_frame[biggest_frame['driver_to_job_time'] < biggest_frame['driver_home_to_job_midpoint_time_limit']]

In [630]:
first_wave = first_wave[first_wave['driver_to_job_distance']>0]
first_wave = first_wave.reset_index()


In [631]:
driver_to_order = {}
for index, row in first_wave.iterrows():
    key = (str(row['terminals']), str(row['order_id']))
    value = my_time(row['driver_to_job_distance_plus'])
    driver_to_order[key] = value

In [632]:
biggest_frame['job_to_driver_distance'] = biggest_frame.apply(lambda row: haversine_distance(row['origin_lng'], row['origin_lat'], row['depot_lng'], row['depot_lat']),axis = 1)

In [633]:
equal_day = []
for index, row in df2.iterrows():
    if row['delivery_start_date_time'].day != row['delivery_end_date_time'].day:
        equal_day.append('1')
    else:
        equal_day.append('0')


In [634]:
df2['same_day'] = equal_day

Examining the computed midpoint time 

In [635]:
#for index, row in df2[df2['same_day']=='1'].iterrows():
 #   print(row['delivery_start_date_time'], row['delivery_start_date_time'] + (row['delivery_end_date_time'] - row['delivery_start_date_time']) /2, row['delivery_end_date_time'])

In [636]:
df2['delivery_midpoint'] = df2['delivery_start_date_time'] + (df2['delivery_end_date_time'] - df2['delivery_start_date_time']) /2

In [637]:
df2['pickup_midpoint'] = df2['pickup_start_date_time'] + (df2['pickup_end_date_time'] - df2['pickup_start_date_time']) /2

In [638]:
zips.columns = ['zip','origin_lat','origin_lng']
df2 = pd.merge(df2, zips, right_on = 'zip', left_on = 'origin_zipcode', how = 'left')
zips.columns = ['zip','dest_lat','dest_lng']
df2 = pd.merge(df2, zips, right_on = 'zip', left_on = 'dest_zipcode', how = 'left')

In [639]:
df2 = df2.drop_duplicates()

In [640]:
df2[df2['order_id'] == 3786037]

,index,order_id,origin_zipcode,dest_zipcode,pickup_window_start,pickup_window_end,delivery_window_start,delivery_window_end,LineHaulRevenue,DispatchMiles,...,monday_6am,driver_home_to_job_midpoint_time_limit,key,same_day,zip_x,origin_lat,origin_lng,zip_y,dest_lat,dest_lng


In [641]:
df2[df2['order_id'] == 3786037]

,index,order_id,origin_zipcode,dest_zipcode,pickup_window_start,pickup_window_end,delivery_window_start,delivery_window_end,LineHaulRevenue,DispatchMiles,...,monday_6am,driver_home_to_job_midpoint_time_limit,key,same_day,zip_x,origin_lat,origin_lng,zip_y,dest_lat,dest_lng


In [642]:
df3 = pd.merge(df2, df2, on = 'key')
df3.columns

Index(['index_x', 'order_id_x', 'origin_zipcode_x', 'dest_zipcode_x',
       'pickup_window_start_x', 'pickup_window_end_x',
       'delivery_window_start_x', 'delivery_window_end_x', 'LineHaulRevenue_x',
       'DispatchMiles_x', 'hour_difference_x', 'pickup_start_date_time_x',
       'pickup_end_date_time_x', 'pickup_midpoint_x',
       'delivery_start_date_time_x', 'delivery_end_date_time_x',
       'delivery_midpoint_x', 'monday_6am_x',
       'driver_home_to_job_midpoint_time_limit_x', 'key', 'same_day_x',
       'zip_x_x', 'origin_lat_x', 'origin_lng_x', 'zip_y_x', 'dest_lat_x',
       'dest_lng_x', 'index_y', 'order_id_y', 'origin_zipcode_y',
       'dest_zipcode_y', 'pickup_window_start_y', 'pickup_window_end_y',
       'delivery_window_start_y', 'delivery_window_end_y', 'LineHaulRevenue_y',
       'DispatchMiles_y', 'hour_difference_y', 'pickup_start_date_time_y',
       'pickup_end_date_time_y', 'pickup_midpoint_y',
       'delivery_start_date_time_y', 'delivery_end_date_time

In [643]:
df3['inter_job_distance'] = df3.apply(lambda row: haversine_distance(row['dest_lng_x'],row['dest_lat_x'],row['origin_lng_y'],row['origin_lat_y']), axis = 1)

In [644]:
df3 = df3[df3['inter_job_distance'] != 0]

In [645]:
# df3['inter_job_distance'].apply(lambda row: my_time(row))
inter_job_time = []
for index, row in df3.iterrows():
    inter_job_time.append(my_time(row['inter_job_distance']))
df3['inter_job_time'] = inter_job_time

In [646]:
df3['inter_job_time_limit'] = (df3['pickup_midpoint_y'] - df3['delivery_midpoint_x']).astype('timedelta64[h]') 

In [647]:
df4 = df3[df3['inter_job_time'] < df3['inter_job_time_limit']]

 ## Creating arcs

In [648]:
import pickle

Purple arcs

In [649]:
# first_wave = first_wave.reset_index()

In [650]:
purple_out_arcs = {}
for index, row in first_wave.iterrows():
    key = (str(row['terminals']), str(row['order_id']))
    value = my_time(row['driver_to_job_distance_plus'])
    purple_out_arcs[key] = value

In [651]:
purple_out_flow_arcs = {}
for index, row in first_wave.iterrows():
    key = (str(row['driver_id']),str(row['terminals']), str(row['order_id']))
    value = my_time(row['driver_to_job_distance_plus'])
    purple_out_flow_arcs[key] = value

In [652]:
purple_in_arcs = {}

for i in range(0,len(first_wave)):
    purple_in_arcs[(str(str(first_wave['order_id'][i])),first_wave['terminals'][i])] = my_time(first_wave["driver_to_job_distance"][i])
    

In [653]:
purple_in_flow_arcs = {}

for index, row in first_wave.iterrows():
    key = (str(row['driver_id']), str(row['order_id']), str(row['terminals']))
    value = my_time(row["driver_to_job_distance"])
    purple_in_flow_arcs[key] = value



green arcs

In [654]:
df4 = df4.reset_index()

In [655]:
df4['second_job_distance'] = df4.apply(lambda row: haversine_distance(row['dest_lng_y'], row['dest_lat_y'], row['origin_lng_y'], row['origin_lat_y']),axis = 1)
df5 = df4[df4['second_job_distance'] > 0]
df5['second_job_time'] = df5['second_job_distance'].apply(lambda row: my_time(row))

In [656]:
df5['green_arc_time'] = df5['second_job_time']+df5['inter_job_time']

In [657]:
df5 = df5.reset_index()
green_arcs = {}

for i in range(0,len(df5)):
    green_arcs[str(df5['order_id_x'][i]),str(df5["order_id_y"][i])] = df5["green_arc_time"][i]

In [658]:
df6 = df5[['order_id_x','order_id_y','green_arc_time']]
df7 = first_wave[['driver_id','order_id']]

In [659]:
df8 = pd.merge(df6,df7,left_on = 'order_id_x', right_on = 'order_id')

In [660]:
df8 = df8.reset_index()
green_flow_arcs = {}

for i in range(0,len(df8 )):
    green_flow_arcs[str(df8['driver_id'][i]), str(df8['order_id_x'][i]),str(df8["order_id_y"][i])] = df8["green_arc_time"][i]

terminals, drivers, and orders

In [661]:
print(len(green_flow_arcs))
print(len(green_arcs))
print(len(purple_out_flow_arcs))
print(len(purple_out_arcs))
print(len(purple_in_arcs))
print(len(purple_in_flow_arcs))

29607
2612
2800
2800
2800
2800


In [662]:
all_arcs = {}
all_arcs.update(purple_out_arcs)
all_arcs.update(purple_in_arcs)
all_arcs.update(green_arcs)

In [663]:
flow_arcs = {}
flow_arcs.update(purple_out_flow_arcs)
flow_arcs.update(purple_in_flow_arcs)
flow_arcs.update(green_flow_arcs)

In [664]:
jobs_revenues = {}

for i in range(0,len(df2)):
    jobs_revenues[str(df2['order_id'][i])] = df2["LineHaulRevenue"][i]

In [665]:
drivers = list(ds['driver_id'])
jobs = list(df2['order_id'].astype(str))
nodes = terminals + jobs

In [666]:
order_data_set = df2

In [667]:
model_materials = (terminals, drivers, jobs, nodes, jobs_revenues, all_arcs, flow_arcs, order_data_set)

In [668]:
for obj in model_materials[:7]:
    if isinstance(obj, dict):
        first_five_items = {k: obj[k] for k in list(obj.keys())[:6]}
    else:
        first_five_items = obj[:6]
    print(first_five_items)

['T0', 'T1', 'T2', 'T3', 'T4', 'T5']
['Driver286', 'Driver223', 'Driver311', 'Driver458', 'Driver378', 'Driver247']
['3305', '476', '4911', '9245', '1438', '3850']
['T0', 'T1', 'T2', 'T3', 'T4', 'T5']
{'3305': 2778.652284813813, '476': 5112.505514467786, '4911': 1427.9496197121769, '9245': 5165.814081230233, '1438': 1490.0483507566723, '3850': 3843.1429844719055}
{('T17', '2363'): 28.437503646537525, ('T29', '2363'): 27.330132230242057, ('T22', '2363'): 23.613028522995, ('T20', '2363'): 26.15496421823642, ('T25', '2363'): 27.20856293091873, ('T0', '2363'): 34.766723710145314}
{('Driver41', 'T17', '2363'): 28.437503646537525, ('Driver331', 'T29', '2363'): 27.330132230242057, ('Driver91', 'T22', '2363'): 23.613028522995, ('Driver228', 'T20', '2363'): 26.15496421823642, ('Driver344', 'T25', '2363'): 27.20856293091873, ('Driver286', 'T0', '2363'): 34.766723710145314}


In [669]:
with open('C:/Users/jiadiz/Desktop/PGT Trucking/initial model/Github/model_materials.pkl', 'wb') as f:
    pickle.dump(model_materials, f)